TLC Trip Record Data 출처: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("spark-sql").getOrCreate()

In [ ]:
# 2015-summary.json

In [2]:
df = spark.read.format('json')\
        .load("learning_spark_data/2015-summary.json")

In [3]:
df.count()

256

In [4]:
df.dtypes

[('DEST_COUNTRY_NAME', 'string'),
 ('ORIGIN_COUNTRY_NAME', 'string'),
 ('count', 'bigint')]

In [5]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [6]:
df.collect()

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Grenada', count=62),
 Row(DEST_COUNTRY_NAME='Costa Rica', ORIGIN_COUNTRY_NAME='United States', count=588),
 Row(DEST_COUNTRY_NAME='Senegal', ORIGIN_COUNTRY_NAME='United States', count=40),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Sint Maarten', count=325),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Marshall Islands', count=39),
 

In [7]:
df.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [8]:
df.select('count').show(5)

+-----+
|count|
+-----+
|   15|
|    1|
|  344|
|   15|
|   62|
+-----+
only showing top 5 rows



In [9]:
# 도착 국가명 중복제거
df1 = df.select('DEST_COUNTRY_NAME').distinct().cache()
df1.show()

+--------------------+
|   DEST_COUNTRY_NAME|
+--------------------+
|            Anguilla|
|              Russia|
|            Paraguay|
|             Senegal|
|              Sweden|
|            Kiribati|
|              Guyana|
|         Philippines|
|            Djibouti|
|            Malaysia|
|           Singapore|
|                Fiji|
|              Turkey|
|                Iraq|
|             Germany|
|              Jordan|
|               Palau|
|Turks and Caicos ...|
|              France|
|              Greece|
+--------------------+
only showing top 20 rows



In [10]:
df1.count()

132

In [11]:
# ROW class를 이용한 단일 레코드 생성

from pyspark.sql import Row
myRow = Row('hello', None, 1, False)
myRow

<Row('hello', None, 1, False)>

In [12]:
# 새로운 컬럼 추가하기
from pyspark.sql.functions import expr # 표현식
df3 = df.withColumn('withinCountry', expr('ORIGIN_COUNTRY_NAME==DEST_COUNTRY_NAME')) # expr sql 표현식을 받아 생성
df3.show(3)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
|    United States|            Ireland|  344|        false|
+-----------------+-------------------+-----+-------------+
only showing top 3 rows



In [13]:
df3.filter(df3.withinCountry == True).show()

+-----------------+-------------------+------+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|withinCountry|
+-----------------+-------------------+------+-------------+
|    United States|      United States|370002|         true|
+-----------------+-------------------+------+-------------+



In [14]:
# case when 카운트 10 이하이면 under, 이상 upper로 변환 > category 컬럼 추가
df4 = df3.withColumn('category', expr("CASE WHEN count <= 10 THEN 'under' ELSE 'upper' END"))
df4.show(5)

+-----------------+-------------------+-----+-------------+--------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|category|
+-----------------+-------------------+-----+-------------+--------+
|    United States|            Romania|   15|        false|   upper|
|    United States|            Croatia|    1|        false|   under|
|    United States|            Ireland|  344|        false|   upper|
|            Egypt|      United States|   15|        false|   upper|
|    United States|              India|   62|        false|   upper|
+-----------------+-------------------+-----+-------------+--------+
only showing top 5 rows



In [15]:
# DataFrame의 select(), where(), filter() 트랜스포메이션
# show(), count() 액션

In [45]:
# emp_df, dept_df
emp_df = spark.read.format("csv") \
    .option("header", True) \
    .option('inferSchema', 'true')\
    .load("learning_spark_data/emp.csv")

dept_df = spark.read.format("csv") \
    .option("header", True) \
    .option('inferSchema', 'true')\
    .load("learning_spark_data/dept.csv")


In [ ]:
emp_df.dtypes

In [46]:
emp_df.show(3)
emp_df.count()

+-----+-----+--------+----+----------+----+----+------+
|empno|ename|     job| mgr|  hiredate| sal|comm|deptno|
+-----+-----+--------+----+----------+----+----+------+
| 7369|SMITH|   CLERK|7902|1980-12-17| 800|NULL|    20|
| 7499|ALLEN|SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7521| WARD|SALESMAN|7698|1981-02-22|1250| 500|    30|
+-----+-----+--------+----+----------+----+----+------+
only showing top 3 rows



15

In [47]:
dept_df.count()
dept_df.show()

+------+----------+--------+
|deptno|     dname|     loc|
+------+----------+--------+
|    10|ACCOUNTING|NEW YORK|
|    20|  RESEARCH|  DALLAS|
|    30|     SALES| CHICAGO|
|    40|OPERATIONS|  BOSTON|
+------+----------+--------+



In [48]:
emp_df.printSchema()
dept_df.printSchema()

root
 |-- empno: integer (nullable = true)
 |-- ename: string (nullable = true)
 |-- job: string (nullable = true)
 |-- mgr: integer (nullable = true)
 |-- hiredate: date (nullable = true)
 |-- sal: integer (nullable = true)
 |-- comm: integer (nullable = true)
 |-- deptno: integer (nullable = true)

root
 |-- deptno: integer (nullable = true)
 |-- dname: string (nullable = true)
 |-- loc: string (nullable = true)



In [49]:
# 컬럼명은 대소문자 구분x
emp_df.select('ENAME', 'DEPTNO').show()

+------+------+
| ENAME|DEPTNO|
+------+------+
| SMITH|    20|
| ALLEN|    30|
|  WARD|    30|
| JONES|    20|
|MARTIN|    30|
| BLAKE|    30|
| CLARK|    10|
| SCOTT|    20|
|  KING|    10|
|TURNER|    30|
| ADAMS|    20|
| JAMES|    30|
|  FORD|    20|
|MILLER|    10|
|  JACK|    70|
+------+------+



In [50]:
# filter()랑 동일함.
emp_df.select('*').where('deptno=20').show()

+-----+-----+-------+----+----------+----+----+------+
|empno|ename|    job| mgr|  hiredate| sal|comm|deptno|
+-----+-----+-------+----+----------+----+----+------+
| 7369|SMITH|  CLERK|7902|1980-12-17| 800|NULL|    20|
| 7566|JONES|MANAGER|7839|1981-04-02|2975|NULL|    20|
| 7788|SCOTT|ANALYST|7566|1987-04-19|3000|NULL|    20|
| 7876|ADAMS|  CLERK|7788|1987-05-23|1100|NULL|    20|
| 7902| FORD|ANALYST|7566|1981-12-03|3000|NULL|    20|
+-----+-----+-------+----+----------+----+----+------+



In [51]:
emp_df.selectExpr('count(*)').show() # 문자열로 count -> sql count

+--------+
|count(1)|
+--------+
|      15|
+--------+



In [52]:
from pyspark.sql.functions import countDistinct
emp_df.select(countDistinct('job')).show()

+-------------------+
|count(DISTINCT job)|
+-------------------+
|                  5|
+-------------------+



In [53]:
from pyspark.sql.functions import approx_count_distinct
emp_df.select(approx_count_distinct('job', 0.1)).show() # 오류율 0.1

+--------------------------+
|approx_count_distinct(job)|
+--------------------------+
|                         5|
+--------------------------+



In [54]:
# first, last, min, max, sum, average -> (expr : sql 문장x), function 으로 처리  테스트

In [55]:
from pyspark.sql.functions import count
emp_df.select(count('sal')).show()

+----------+
|count(sal)|
+----------+
|        15|
+----------+



In [56]:
from pyspark.sql.functions import first, last
emp_df.select(first('hiredate')).show()
emp_df.select(last('hiredate')).show()

+---------------+
|first(hiredate)|
+---------------+
|     1980-12-17|
+---------------+

+--------------+
|last(hiredate)|
+--------------+
|    1982-01-23|
+--------------+



In [57]:
from pyspark.sql.functions import min, max
emp_df.select(min('sal')).show()
emp_df.select(max('sal')).show()

+--------+
|min(sal)|
+--------+
|     800|
+--------+

+--------+
|max(sal)|
+--------+
|    5000|
+--------+



In [58]:
from pyspark.sql.functions import sum
emp_df.select(sum('comm')).show()

+---------+
|sum(comm)|
+---------+
|     2200|
+---------+



In [59]:
from pyspark.sql.functions import avg, round
emp_df.select(round(avg('sal'), 2).alias('avg_sal')).show()

+-------+
|avg_sal|
+-------+
|2148.33|
+-------+



In [60]:
from pyspark.sql.functions import countDistinct
emp_df.select(countDistinct('deptno')).show()

+----------------------+
|count(DISTINCT deptno)|
+----------------------+
|                     4|
+----------------------+



In [61]:
from pyspark.sql.functions import col
emp_df.select(sum(col('sal'))).show()

+--------+
|sum(sal)|
+--------+
|   32225|
+--------+



In [62]:
emp_df.selectExpr('sum(distinct sal)').show()

+-----------------+
|sum(DISTINCT sal)|
+-----------------+
|            27975|
+-----------------+



In [63]:
# total_salary / total_transaction, avg_salary, mean_salary

In [64]:
from pyspark.sql.functions import sum, count, avg

emp_df.select(
    sum("sal").alias("total_salary"),
    count("sal").alias("total_transaction"),
    (sum("sal") / count("sal")).alias("avg_salary"),
    avg("sal").alias("mean_salary")
).selectExpr(
    'total_salary / total_transaction',
    'avg_salary',
    'mean_salary'
).show()

+----------------------------------+------------------+------------------+
|(total_salary / total_transaction)|        avg_salary|       mean_salary|
+----------------------------------+------------------+------------------+
|                2148.3333333333335|2148.3333333333335|2148.3333333333335|
+----------------------------------+------------------+------------------+



In [65]:
# 그룹화
emp_df.groupBy('job').count().show()

+---------+-----+
|      job|count|
+---------+-----+
|  ANALYST|    2|
| SALESMAN|    4|
|    CLERK|    5|
|  MANAGER|    3|
|PRESIDENT|    1|
+---------+-----+



In [66]:
# select job,
#     count(job),
#     sum(sal)
# group by job

group_df = emp_df.groupBy('job').agg(
    count('job').alias('qty'),
    expr('count(job)'),
    sum('sal')
)
group_df.show()

+---------+---+----------+--------+
|      job|qty|count(job)|sum(sal)|
+---------+---+----------+--------+
|  ANALYST|  2|         2|    6000|
| SALESMAN|  4|         4|    5600|
|    CLERK|  5|         5|    7350|
|  MANAGER|  3|         3|    8275|
|PRESIDENT|  1|         1|    5000|
+---------+---+----------+--------+



In [67]:
# sal의 평균 SAL_AVG, 표준편차 SAL_STDEV를 job별로 계산해서 출력, 소수점 2자리
from pyspark.sql.functions import avg, stddev, round
emp_df.groupBy('job').agg(
    round(avg('sal'),2).alias('SAL_AVG'),
    round(stddev('sal'),2).alias('SAL_STDEV')
).show()

+---------+-------+---------+
|      job|SAL_AVG|SAL_STDEV|
+---------+-------+---------+
|  ANALYST| 3000.0|      0.0|
| SALESMAN| 1400.0|   177.95|
|    CLERK| 1470.0|   984.63|
|  MANAGER|2758.33|   274.24|
|PRESIDENT| 5000.0|     NULL|
+---------+-------+---------+



In [68]:
# 급여 top10 구하기
emp_df.orderBy(emp_df.sal.desc()).limit(10).show()

+-----+------+---------+----+----------+----+----+------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|
| 9292|  JACK|    CLERK|7782|1982-01-23|3200|NULL|    70|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|
+-----+------+---------+----+----------+----+----+------+



In [69]:
#윈도우 함수
from pyspark.sql.functions import desc, rank
from pyspark.sql.window import Window
windowspec = Window.orderBy(desc('sal'))
salAllRank = rank().over(windowspec)
salAllRank

Column<'RANK() OVER (ORDER BY sal DESC NULLS LAST unspecifiedframe$())'>

In [70]:
emp_df.show()

+-----+------+---------+----+----------+----+----+------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|NULL|    20|
| 7900| JAMES|    CLERK|7698|1981-12-03| 950|NULL|    30|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|
| 9292|  JACK|

In [71]:
emp_df.withColumn('salary_rank', salAllRank).show(10)

+-----+------+---------+----+----------+----+----+------+-----------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|salary_rank|
+-----+------+---------+----+----------+----+----+------+-----------+
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|          1|
| 9292|  JACK|    CLERK|7782|1982-01-23|3200|NULL|    70|          2|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|          3|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|          3|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|          5|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|          6|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|          7|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|          8|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|          9|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|         10|
+-----+------+---------+----+----------+----+----+------+-----------+
only showing top 10 

In [72]:
# 직무별로 rank 작성
# Window.partitionBy()
# job_rank_df 작성

In [76]:
job_window = Window.partitionBy('job').orderBy(emp_df.sal.desc())
job_rank_df = emp_df.withColumn('rank', rank().over(job_window))
job_rank_df.orderBy('job','rank').show()

+-----+------+---------+----+----------+----+----+------+----+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|rank|
+-----+------+---------+----+----------+----+----+------+----+
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|   1|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|   1|
| 9292|  JACK|    CLERK|7782|1982-01-23|3200|NULL|    70|   1|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|   2|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|NULL|    20|   3|
| 7900| JAMES|    CLERK|7698|1981-12-03| 950|NULL|    30|   4|
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|   5|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|   1|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|   2|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|   3|
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|   1|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|   1|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    3

In [ ]:
# 부서별 순위

In [78]:
dept_window = Window.partitionBy("deptno").orderBy(desc("sal"))
dept_rank_df = emp_df.withColumn("dept_rank", rank().over(dept_window))
dept_rank_df.orderBy("deptno", "dept_rank").show()

+-----+------+---------+----+----------+----+----+------+---------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|dept_rank|
+-----+------+---------+----+----------+----+----+------+---------+
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|        1|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|        2|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|        3|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|        1|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|        1|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|        3|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|NULL|    20|        4|
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|        5|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|        1|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|        2|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|        3|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 50

In [ ]:
# 부서별 누적급여 sum('sal').over()

In [84]:
dept_window = Window.partitionBy("deptno").orderBy("sal") \
                   .rowsBetween(Window.unboundedPreceding, Window.currentRow) # 그룹의 처음행부터 현재행까지 누적

emp_df.withColumn("cumulative_sal", sum("sal").over(dept_window)).show() # 누적 합계 계산

+-----+------+---------+----+----------+----+----+------+--------------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|cumulative_sal|
+-----+------+---------+----+----------+----+----+------+--------------+
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|          1300|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|          3750|
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|          8750|
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|           800|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|NULL|    20|          1900|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|          4875|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|          7875|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|         10875|
| 7900| JAMES|    CLERK|7698|1981-12-03| 950|NULL|    30|           950|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|          2200|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400| 

In [ ]:
# 부서별 평균급여와 직원 개별 급여 비교

In [85]:
# 부서별 평균 급여
dept_avg_window = Window.partitionBy("deptno")
emp_df_with_avg = emp_df.withColumn("avg_sal", avg("sal").over(dept_avg_window))

In [91]:
from pyspark.sql.functions import when, col
emp_compare_df = emp_df_with_avg.withColumn(
    "salary_comparison",
    when(col("sal") > col("avg_sal"), "+")
    .when(col("sal") < col("avg_sal"), "-")
    .otherwise("=")
)

In [92]:
emp_compare_df.select("deptno", "ename", "sal", "avg_sal", "salary_comparison").show()

+------+------+----+------------------+-----------------+
|deptno| ename| sal|           avg_sal|salary_comparison|
+------+------+----+------------------+-----------------+
|    10| CLARK|2450|2916.6666666666665|                -|
|    10|  KING|5000|2916.6666666666665|                +|
|    10|MILLER|1300|2916.6666666666665|                -|
|    20| SMITH| 800|            2175.0|                -|
|    20| JONES|2975|            2175.0|                +|
|    20| SCOTT|3000|            2175.0|                +|
|    20| ADAMS|1100|            2175.0|                -|
|    20|  FORD|3000|            2175.0|                +|
|    30| ALLEN|1600|1566.6666666666667|                +|
|    30|  WARD|1250|1566.6666666666667|                -|
|    30|MARTIN|1250|1566.6666666666667|                -|
|    30| BLAKE|2850|1566.6666666666667|                +|
|    30|TURNER|1500|1566.6666666666667|                -|
|    30| JAMES| 950|1566.6666666666667|                -|
|    70|  JACK

In [93]:
# 부서별 직업별 소계

In [ ]:
emp_df.groupBy('depno', 'job').agg(count('*'), sum(

In [95]:
emp_df.cube('deptno', 'job').agg(count('*'), sum('sal'))\
    .orderBy('deptno', 'job').show()

+------+---------+--------+--------+
|deptno|      job|count(1)|sum(sal)|
+------+---------+--------+--------+
|  NULL|     NULL|      15|   32225|
|  NULL|  ANALYST|       2|    6000|
|  NULL|    CLERK|       5|    7350|
|  NULL|  MANAGER|       3|    8275|
|  NULL|PRESIDENT|       1|    5000|
|  NULL| SALESMAN|       4|    5600|
|    10|     NULL|       3|    8750|
|    10|    CLERK|       1|    1300|
|    10|  MANAGER|       1|    2450|
|    10|PRESIDENT|       1|    5000|
|    20|     NULL|       5|   10875|
|    20|  ANALYST|       2|    6000|
|    20|    CLERK|       2|    1900|
|    20|  MANAGER|       1|    2975|
|    30|     NULL|       6|    9400|
|    30|    CLERK|       1|     950|
|    30|  MANAGER|       1|    2850|
|    30| SALESMAN|       4|    5600|
|    70|     NULL|       1|    3200|
|    70|    CLERK|       1|    3200|
+------+---------+--------+--------+



In [96]:
dept_df.printSchema()

root
 |-- deptno: integer (nullable = true)
 |-- dname: string (nullable = true)
 |-- loc: string (nullable = true)



In [98]:
emp_dept_df = emp_df.join(dept_df, emp_df['deptno']==dept_df['deptno'])
emp_dept_df.show()

+-----+------+---------+----+----------+----+----+------+------+----------+--------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|deptno|     dname|     loc|
+-----+------+---------+----+----------+----+----+------+------+----------+--------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|    20|  RESEARCH|  DALLAS|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|    30|     SALES| CHICAGO|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|    30|     SALES| CHICAGO|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|    20|  RESEARCH|  DALLAS|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|    30|     SALES| CHICAGO|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|    30|     SALES| CHICAGO|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|    10|ACCOUNTING|NEW YORK|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|    20|  RESEARCH|  DALLAS|
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|    10|A

In [100]:
join_df = emp_df.join(dept_df, on='deptno',how='inner')
join_df.select('ename','deptno','dname').show()

+------+------+----------+
| ename|deptno|     dname|
+------+------+----------+
| SMITH|    20|  RESEARCH|
| ALLEN|    30|     SALES|
|  WARD|    30|     SALES|
| JONES|    20|  RESEARCH|
|MARTIN|    30|     SALES|
| BLAKE|    30|     SALES|
| CLARK|    10|ACCOUNTING|
| SCOTT|    20|  RESEARCH|
|  KING|    10|ACCOUNTING|
|TURNER|    30|     SALES|
| ADAMS|    20|  RESEARCH|
| JAMES|    30|     SALES|
|  FORD|    20|  RESEARCH|
|MILLER|    10|ACCOUNTING|
+------+------+----------+



In [101]:
spark.stop()